In [1]:
%load_ext autoreload
%autoreload 2
import sys
if "../" not in sys.path:
    sys.path.append("../")
import os
import json

import datetime
import matplotlib

from volterra_hawkes.kernel.kernels import *
from volterra_hawkes.utility.visualisation import Experiment, plot_kernel

from notebooks.numerical_tests import *

plt.style.use('seaborn-v0_8-deep')
plt.style.use('default')  # 'default' or any other light theme like 'seaborn-white'

color_cycle = ["#B56246", "#579F40", "#9A46B5", "#4699B5", "#B54662", "#D4A017", "#5b6c64", "#71074E", "#0033A0"]
plt.rcParams["figure.figsize"]        = ((1 + np.sqrt(5)) / 2 * 5, 5)
plt.rcParams["figure.autolayout"]     = True
plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams["axes.grid"]             = True
plt.rcParams['axes.prop_cycle']       = matplotlib.cycler(color=color_cycle)

In [2]:
lam_gamma_3 = 3
alpha_gamma_3 = 2
t0_gamma_3 = (alpha_gamma_3 - 1) / lam_gamma_3
gamma_kernel_3 = GammaKernel(lam=lam_gamma_3, alpha=alpha_gamma_3, c=0.9 * lam_gamma_3 ** alpha_gamma_3)

lam_gamma_2 = 10
alpha_gamma_2 = 3
t0_gamma_2 = (alpha_gamma_2 - 1) / lam_gamma_2
gamma_kernel_2 = GammaKernel(lam=lam_gamma_2, alpha=alpha_gamma_2, c=0.95 * lam_gamma_2 ** alpha_gamma_2)

experiments_list = [
    Experiment(
    T=1,
    n_steps = 1000,
    kernel = ExponentialKernel(c=4, lam=5),
    mu=10
    ),
    Experiment(
    T=1,
    n_steps=100,
    kernel=gamma_kernel_2,
    mu=10,
    decreasing_kernel=lambda t: gamma_kernel_2(t) * (t > t0_gamma_2) + gamma_kernel_2(t0_gamma_2) * (t <= t0_gamma_2),
    ),
    Experiment(
    T=1,
    n_steps = 1000,
    kernel = FractionalKernel(H=0.1, c=0.8),
    mu=10
    ),
    Experiment(
    T=1,
    n_steps=100,
    kernel=gamma_kernel_3,
    mu=5,
    decreasing_kernel=lambda t: gamma_kernel_3(t) * (t > t0_gamma_3) + gamma_kernel_3(t0_gamma_3) * (t <= t0_gamma_3),
    ),
]

In [3]:
scaling_factors = [10] # [5, 3, 7]

experiments_list_gamma = []

for scaling_factor in scaling_factors:
    lam_gamma_sc = 3 * scaling_factor
    alpha_gamma_sc = 2
    experiment = Experiment(
        T=1,
        n_steps=1000,
        kernel=GammaKernel(lam=lam_gamma_sc, alpha=alpha_gamma_sc, c=0.9 * lam_gamma_sc ** alpha_gamma_sc),
        mu=5 * scaling_factor,
    )
    experiments_list_gamma.append(experiment)

In [4]:
scaling_factors = [1, 2] # [5, 1, 3, 7]

experiments_list_testing_exp = []

for scaling_factor in scaling_factors:
    experiment = Experiment(
    T=1,
    n_steps = 1000,
    kernel = ExponentialKernel(c=4 * scaling_factor, lam=5 * scaling_factor),
    mu=10 * scaling_factor
    )
    experiments_list_testing_exp.append(experiment)

In [5]:
experiments_fract = [
    Experiment(
    T=1,
    n_steps = 1000,
    kernel = FractionalKernel(H=0.1, c=0.1 * scale**(0.1 + 0.5)),
    mu=5 * scale
    )
    for scale in [5, 10, 20, 30]]

In [6]:
methods_non_ivi = ["Population", "Ogata"]
methods_ivi = ["iVi"] # "Res iVi"
n_paths_time_meas = 10**5
batch_size_cf = 10**5
n_batch_cf = 10
n_steps_arr_cf = [20, 35, 50, 65, 80, 100, 200, 350, 500, 750, 1000]

def save_results(path_experiment, experiment_results):
    # saving the results
    with open(path_experiment + 'results.json', 'w') as f:
        json.dump(experiment_results, f)

path = "../experiments/"

for e in (experiments_list_gamma):
    print("RUNNING EXPERIMENT", str(e))
    try:
        name = str(e)[:str(e).index("decreasing") - 2] + ")"
    except ValueError:
        name = str(e)
    name = name + "_" + datetime.datetime.now().strftime('%Y-%m-%d_%H_%M_%S')
    path_experiment = path + name + "/"
    try:
        os.mkdir(path_experiment)
    except FileExistsError as _:
        pass

    experiment_results = dict()
    experiment_results["methods_ivi"] = methods_ivi
    experiment_results["methods_non_ivi"] = methods_non_ivi
    experiment_results["n_paths_time_meas"] = n_paths_time_meas
    experiment_results["n_steps_arr_cf"] = n_steps_arr_cf
    experiment_results["n_batch_cf"] = n_batch_cf
    experiment_results["batch_size_cf"] = batch_size_cf

    print("Pre-computing non-iVi samples...")
    try:
        experiment_results["n_paths_non_ivi"] = 10_000
        samples_non_ivi = {}
        for method in methods_non_ivi:
            time_non_ivi = time.time()
            samples_non_ivi[method] = get_N_U_sample(experiment=e, method=method,
                                                     n_paths=experiment_results["n_paths_non_ivi"],
                                                     return_counters=True)
            time_non_ivi = time.time() - time_non_ivi
            experiment_results["time_" + method] = time_non_ivi * (experiment_results["n_paths_time_meas"] /
                                                                   experiment_results["n_paths_non_ivi"])
    except Exception as e:
        print("Exception occured during computing non-iVI samples: ", e)
        continue

    # print("RUNNING TEST 0, EXPERIMENT", str(e))
    # run_test0(e, path_experiment, experiment_results)
    # save_results(path_experiment, experiment_results)
    # print("RUNNING TEST 1, EXPERIMENT", str(e))
    # run_test1(e, path_experiment, experiment_results)
    # save_results(path_experiment, experiment_results)
    # print("RUNNING TEST 2, EXPERIMENT", str(e))
    # run_test2(e, path_experiment, experiment_results, samples_non_ivi)
    # save_results(path_experiment, experiment_results)
    # print("RUNNING TEST 3, EXPERIMENT", str(e))
    # run_test3(e, path_experiment, experiment_results, samples_non_ivi)
    # save_results(path_experiment, experiment_results)
    print("RUNNING TEST 4, EXPERIMENT", str(e))
    run_test4(e, path_experiment, experiment_results)
    save_results(path_experiment, experiment_results)

    plt.close('all')

RUNNING EXPERIMENT Experiment(T=1, n_steps=1000, kernel=GammaKernel(alpha=2, lam=30, c=810.0), mu=50, decreasing_kernel=<function Experiment.__post_init__.<locals>.decreasing_kernel at 0x000002042436E790>)
Pre-computing non-iVi samples...
RUNNING TEST 4, EXPERIMENT Experiment(T=1, n_steps=1000, kernel=GammaKernel(alpha=2, lam=30, c=810.0), mu=50, decreasing_kernel=<function Experiment.__post_init__.<locals>.decreasing_kernel at 0x000002042436E790>)
Population
Kolmogorov-Smirnov test p-value:  0.6898771332856012
Ogata
Kolmogorov-Smirnov test p-value:  0.5787178561624456
iVi
Kolmogorov-Smirnov test p-value:  0.3696158938238343
Population
Kolmogorov-Smirnov test p-value:  0.6898771332856012
Ogata
Kolmogorov-Smirnov test p-value:  0.5787178561624456
Exception occured: operands could not be broadcast together with shapes (393,) (392,) 
Population
Kolmogorov-Smirnov test p-value:  0.6898771332856012
Ogata
Kolmogorov-Smirnov test p-value:  0.5787178561624456
iVi
Kolmogorov-Smirnov test p-valu